# **NLP1_Run3**

In [1]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.4/325.4 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 71.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.6 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [2]:

import openai
import os
from IPython.display import Markdown
import pyomo


### **Accessing the GPT4 API**

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']



### **Generate Mathematical Model**

In [4]:
problem = """ A firm that packs refreshments and beers, situated in the province of Valencia (Spain) employs the same syrup to produce its 1.5 l COLI and PEPSA products on its S1 production line. Once processed, each hectolitre of syrup produces 40 units of the 1.5 l COLI product and 20 units of the 1.5 l PEPSA product. If X1 units of the 1.5 l COLI product and X2 units of the 1.5 l PEPSA product are produced, the firm estimates that the daily income obtained in dollars would be given by the following function:
49000 times X1 minus X1 squared plus 30 times X2 minus two times X2 squared.
It costs 150 dollars to buy and process each hectolitre of syrup. The S1 packaging line has a net capacity of producing 7100 1.5 l product units every hour. The firm works 5 days a week in 8h shifts. Given its weekly target coverage, the firm is committed to produce at least half the amount of PEPSA than COLI. Although priority orders tend to amend its production planning, the firm wishes to have a basic product planning that optimises its daily profits. """

In [5]:
client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 3
)


In [6]:
print(response1.choices[0].message.content)

Parameters:

- C1 = Cost of syrup per hectolitre = $150
- P1 = Production of COLI per hectolitre of syrup = 40 units
- P2 = Production of PEPSA per hectolitre of syrup = 20 units
- DC1 = Daily capacity of processing COLI = 7100 Gb * 8 hours * 5 days = 284000 units
- DC2 = Daily capacity of processing PEPSA = 7100 Gb * 8 hours * 5 days = 284000 units
- IC1 = Income function of COLI = 49000*X1 - X1^2
- IC2 = Income function of PEPSA = 30*X2 - 2*X2^2
- W = Working days per week = 5 days
- S = Working shifts per week = 8 hours

Decision Variables:

- X1 = Units of COLI to be produced
- X2 = Units of PEPSA to be produced

Objective Function:

The firm is looking to maximise its daily income so the objective function is:

Max Z = IC1 + IC2 - (C1 *(X1/P1) + C1 *(X2/P2))

Constraints:

- Capacity constraints :

X1 <= DC1
X2 <= DC2

- Production Ratio constraint :

X2 >= 0.5*X1

- Non-negativity constraints :

X1 >= 0
X2 >= 0


### **Generate Pyomo Code**

In [7]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 3
)

In [8]:
Markdown(response2.choices[0].message.content)

Here is a Python pyomo code for the given optimization problem:

```Python
from pyomo.environ import *

# Create a simple model
model = ConcreteModel()

# Define parameters
# Note: These are the sample data
model.C1 = Param(initialize=150)    # Cost of syrup per hectolitre
model.P1 = Param(initialize=40)     # Production of COLI per hectolitre of syrup
model.P2 = Param(initialize=20)     # Production of PEPSA per hectolitre of syrup
model.DC1 = Param(initialize=284000)  # Daily capacity of processing COLI
model.DC2 = Param(initialize=284000)  # Daily capacity of processing PEPSA

# Define decision variables
model.X1 = Var(within=NonNegativeReals)   # Units of COLI to be produced
model.X2 = Var(within=NonNegativeReals)   # Units of PEPSA to be produced

# Define Income functions. These are part of the objective function
def IC1(model):
    return 49000*model.X1 - model.X1**2
model.IC1 = Expression(rule=IC1)

def IC2(model):
    return 30*model.X2 - 2*model.X2**2
model.IC2 = Expression(rule=IC2)

# Define Objective function to maximize income
def objective(model):
    return model.IC1 + model.IC2 - (model.C1 *(model.X1/model.P1) + model.C1 *(model.X2/model.P2))
model.objective = Objective(rule=objective, sense=maximize)

# Define the constraints
def cola_capacity_rule(model):
    return model.X1 <= model.DC1
model.cola_capacity = Constraint(rule=cola_capacity_rule)

def pepsi_capacity_rule(model):
    return model.X2 <= model.DC2
model.pepsi_capacity = Constraint(rule=pepsi_capacity_rule)

def product_ratio_rule(model):
    return model.X2 >= 0.5*model.X1
model.product_ratio = Constraint(rule=product_ratio_rule)

# Solve the model
solver = SolverFactory('glpk')
solver.solve(model)

# Print the output
print("Optimal solution: ")
print("Units of COLI to be produced = ", model.X1())
print("Units of PEPSA to be produced = ", model.X2())
```

Sample data is being used for the parameters, namely Cost of syrup per hectolitre, Production of COLI and PEPSA per hectolitre of syrup, and Daily capacities of processing COLI and PEPSA. Also, income functions are included as separate expressions for clarity, they can be combined into the objective function directly.

### **Run the code Generated by GPT4**

In [10]:
# Install Python API for AMPL
!pip install amplpy --upgrade

from amplpy import AMPL,ampl_notebook

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 49.6 MB/s eta 0:00:00


In [11]:

_ = load_dotenv(find_dotenv('second_knitro_license.env'))
my_knitro_license = os.environ['knitro_license']

ampl = ampl_notebook(
    modules=["knitro"],  # modules to install
    license_uuid=my_knitro_license,  # license to use
)

Licensed to AMPL Community Edition License for <cevikmelis14@gmail.com>.


In [13]:
from pyomo.environ import *

# Create a simple model
model = ConcreteModel()

# Define parameters
# Note: These are the sample data
model.C1 = Param(initialize=150)    # Cost of syrup per hectolitre
model.P1 = Param(initialize=40)     # Production of COLI per hectolitre of syrup
model.P2 = Param(initialize=20)     # Production of PEPSA per hectolitre of syrup
model.DC1 = Param(initialize=284000)  # Daily capacity of processing COLI
model.DC2 = Param(initialize=284000)  # Daily capacity of processing PEPSA

# Define decision variables
model.X1 = Var(within=NonNegativeReals)   # Units of COLI to be produced
model.X2 = Var(within=NonNegativeReals)   # Units of PEPSA to be produced

# Define Income functions. These are part of the objective function
def IC1(model):
    return 49000*model.X1 - model.X1**2
model.IC1 = Expression(rule=IC1)

def IC2(model):
    return 30*model.X2 - 2*model.X2**2
model.IC2 = Expression(rule=IC2)

# Define Objective function to maximize income
def objective(model):
    return model.IC1 + model.IC2 - (model.C1 *(model.X1/model.P1) + model.C1 *(model.X2/model.P2))
model.objective = Objective(rule=objective, sense=maximize)

# Define the constraints
def cola_capacity_rule(model):
    return model.X1 <= model.DC1
model.cola_capacity = Constraint(rule=cola_capacity_rule)

def pepsi_capacity_rule(model):
    return model.X2 <= model.DC2
model.pepsi_capacity = Constraint(rule=pepsi_capacity_rule)

def product_ratio_rule(model):
    return model.X2 >= 0.5*model.X1
model.product_ratio = Constraint(rule=product_ratio_rule)

# Solve the model
solver = SolverFactory('knitro')
solver.solve(model)

# Print the output
print("Optimal solution: ")
print("Units of COLI to be produced = ", model.X1())
print("Units of PEPSA to be produced = ", model.X2())
print(model.objective())

Optimal solution: 
Units of COLI to be produced =  16335.833419111377
Units of PEPSA to be produced =  8167.916766393859
400289174.18594784


### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**